In [89]:
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv

urls = []

for p in range (1, 21):
    url = f'https://www.wildberries.ru/catalog/knigi/nehudozhestvennaya-literatura/psihologiya?sort=popular&page={p}'

    s = Service('C:/Users/User/Desktop/Python/проект/webdrivers/chromedriver.exe')
    browser = webdriver.Chrome(service=s)                          
    browser.get(url)

    books_divs = []  
    for i in range(11):  
        browser.execute_script("window.scrollTo(1, document.body.scrollHeight);")  
        time.sleep(1) 
        html = browser.page_source 
        soup = BeautifulSoup(html, 'html.parser') 
        books = soup.find_all('div', class_='product-card__wrapper')

    for book in books:
        url = book.find('a', class_='product-card__link j-card-link j-open-full-product-card').get('href')
        urls.append(url)
        
with open('urls.csv', 'w', newline='\n', encoding="utf-8") as file:
        writer = csv.writer(file)
        for url in urls:
            writer.writerow([url])
    
        

In [90]:
urls = []
with open('urls.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        for row in reader:
            if row:  # Проверяем, что строка не пустая
                url = row[0]  
                urls.append(url)
                
with open('final.csv', 'a', newline='\n', encoding='utf-8') as file:
    writer = csv.writer(file)
    for index, url in enumerate(urls, start=1):
        data = [index, url] 

        browser.get(url)
        browser.execute_script("window.scrollTo(1, document.body.scrollHeight);")
        time.sleep(2)
        
        try: 
            book_name = browser.find_element(By.XPATH,'//h1').text
            data.append(book_name)
        except:
            data.append('None')
        
        try: 
            book_price = browser.find_element(By.XPATH,'//ins').text
            data.append(book_price)
        except:
            data.append('None')
            
        try: 
            book_rating_count = browser.find_element(By.XPATH, "//span[@class='product-review__count-review']").text
            data.append(book_rating_count)
        except:
            data.append('None')
            
        try: 
            book_rating = browser.find_element(By.XPATH, "//span[@class='product-review__rating address-rate-mini address-rate-mini--sm']").text
            data.append(book_rating)
        except:
            data.append('None')
            
        # мы пробовали запарсить вес товара, количсетво страниц, но на кждой странице тэг ля них отличается, поэтому цикл омается
    
        writer.writerow(data)
        
file.close()

Кодировка файла была некорректной, вследствие чего подправили ее в Excel. Из-за этого столбцы разделились по колонкам - вернем все к виду csv

In [145]:
# чистим 5-ый столбик
df = pd.read_excel('C:/Users/User/Desktop/Python/проект/finally.xlsx')
df['Number of rating no letters'] = df['Number of rating'].str.replace('\D', '', regex=True).str.replace(' ', '')
df['Number of rating no letters']

0         79
1       1511
2         67
3       2695
4          5
        ... 
2004       7
2005      90
2006       6
2007       4
2008       8
Name: Number of rating no letters, Length: 2009, dtype: object

In [155]:
# доп. обработка Number of rating no letters
df['Number of rating no letters'] = df['Number of rating no letters'].replace('', float('nan'))
df['Number of rating no letters'] = df['Number of rating no letters'].dropna().astype(float)
mean_value = df['Number of rating no letters'].mean()
df['Number of rating no letters'].fillna(mean_value, inplace=True)

0         79.0
1       1511.0
2         67.0
3       2695.0
4          5.0
         ...  
2004       7.0
2005      90.0
2006       6.0
2007       4.0
2008       8.0
Name: Number of rating no letters, Length: 2009, dtype: float64

In [162]:
df['Price'] = df['Price'].replace('', float('nan'))
df['Price'] = df['Price'].dropna().astype(float)
mean_value = df['Number of rating no letters'].mean()
df['Price'].fillna(mean_value, inplace=True)

In [163]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Загружаем данные из файла
number_of_rating = df[['Number of rating no letters']]  # Колонка с количеством отзывов на книги
prices = df['Price']  # Колонка с ценами книг

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(number_of_rating, prices, test_size=0.2, random_state=42)

# Создаем модель линейной регрессии

modelka = LinearRegression()

# Изменяем форму массива весов для соответствия ожидаемым входным данным модели
X = number_of_rating

# Обучаем модель на входных данных
modelka.fit(X_train, y_train)

# Предсказание цены на тестовой выборке
y_pred = modelka.predict(X_test)

# Вывод предсказанных значений и фактических значений
predictions = pd.DataFrame({'Фактические значения': y_test, 'Предсказанные значения': y_pred})
print(predictions)



      Фактические значения  Предсказанные значения
1593                 964.0              748.382542
526                  902.0              745.233734
393                  217.0              745.233734
1403                 292.0              748.075213
433                  178.0              745.233734
...                    ...                     ...
940                  600.0              745.847076
733                  525.0              745.233734
1575                1051.0              748.459374
692                  573.0              745.233734
1255                 456.0              747.767883

[402 rows x 2 columns]


In [ ]:
Ввиду малого количества данных предсказанные значения получились не очень точными